Import Packages

In [1]:
import pandas as pd
import numpy as np
from pyampute.exploration.mcar_statistical_tests import MCARTest
from sklearn.impute import KNNImputer

Read Data and format numbers

In [2]:
columns=['Entity', 'Year', 'Access to electricity (% of population)', 'Access to clean fuels for cooking', 'Renewable-electricity-generating-capacity-per-capita', 'Financial flows to developing countries (US $)', 'Renewable energy share in the total final energy consumption (%)', 'Electricity from fossil fuels (TWh)', 'Electricity from nuclear (TWh)', 'Electricity from renewables (TWh)', 'Low-carbon electricity (% electricity)', 'Primary energy consumption per capita (kWh/person)', 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)', 'Value_co2_emissions_kt_by_country', 'Renewables (% equivalent primary energy)', 'gdp_growth', 'gdp_per_capita', 'Density\\n(P/Km2)', 'Land Area(Km2)', 'Latitude', 'Longitude']

def convert_to_float(value):
    if isinstance(value, str) and ',' in value:
        return float(value.replace(',', '.'))
    return value

data = pd.read_csv('./co2_emissions/emission_data.csv')#, converters={'Density\\n(P/Km2)': convert_to_float})
pd.DataFrame(data)

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.61,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59,1.64,760.0,NaN,NaN,NaN,60.0,652230.0,33.939110,67.709953
1,Afghanistan,2001,4.07,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89,1.74,730.0,NaN,NaN,NaN,60.0,652230.0,33.939110,67.709953
2,Afghanistan,2002,9.41,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86,1.40,1030.0,NaN,NaN,179.43,60.0,652230.0,33.939110,67.709953
3,Afghanistan,2003,14.74,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.97,1.40,1220.0,NaN,8.83,190.68,60.0,652230.0,33.939110,67.709953
4,Afghanistan,2004,20.06,10.9,7.75,NaN,44.24,0.33,0.0,0.56,...,204.23,1.20,1030.0,NaN,1.41,211.38,60.0,652230.0,33.939110,67.709953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,Zimbabwe,2016,42.56,29.8,62.88,30000.0,81.90,3.50,0.0,3.32,...,3227.68,10.00,11020.0,NaN,0.76,1464.59,38.0,390757.0,-19.015438,29.154857
3645,Zimbabwe,2017,44.18,29.8,62.33,5570000.0,82.46,3.05,0.0,4.30,...,3068.01,9.51,10340.0,NaN,4.71,1235.19,38.0,390757.0,-19.015438,29.154857
3646,Zimbabwe,2018,45.57,29.9,82.53,10000.0,80.23,3.73,0.0,5.46,...,3441.99,9.83,12380.0,NaN,4.82,1254.64,38.0,390757.0,-19.015438,29.154857
3647,Zimbabwe,2019,46.78,30.1,81.40,250000.0,81.50,3.66,0.0,4.58,...,3003.66,10.47,11760.0,NaN,-6.14,1316.74,38.0,390757.0,-19.015438,29.154857


Determine Number of zeros per column

In [3]:
num_zeros = (data == 0).sum()*100 / len(data)
print(num_zeros)

Entity                                                               0.000000
Year                                                                 0.000000
Access to electricity (% of population)                              0.000000
Access to clean fuels for cooking                                    0.219238
Renewable-electricity-generating-capacity-per-capita                 6.275692
Financial flows to developing countries (US $)                       0.685119
Renewable energy share in the total final energy consumption (%)     2.603453
Electricity from fossil fuels (TWh)                                  3.864072
Electricity from nuclear (TWh)                                      80.707043
Electricity from renewables (TWh)                                   17.511647
Low-carbon electricity (% electricity)                              16.963552
Primary energy consumption per capita (kWh/person)                   0.575500
Energy intensity level of primary energy (MJ/$2017 PPP GDP)     

Determine the number of NaNs

In [4]:
nan_counts = data.isna().sum() *100 / len(data)
print(nan_counts)

Entity                                                               0.000000
Year                                                                 0.000000
Access to electricity (% of population)                              0.274048
Access to clean fuels for cooking                                    4.631406
Renewable-electricity-generating-capacity-per-capita                25.513839
Financial flows to developing countries (US $)                      57.248561
Renewable energy share in the total final energy consumption (%)     5.316525
Electricity from fossil fuels (TWh)                                  0.575500
Electricity from nuclear (TWh)                                       3.453001
Electricity from renewables (TWh)                                    0.575500
Low-carbon electricity (% electricity)                               1.151000
Primary energy consumption per capita (kWh/person)                   0.000000
Energy intensity level of primary energy (MJ/$2017 PPP GDP)     

Remove Columns with more than 20% zeros as they cannot be good reference for the entire set

In [5]:
data.drop('Electricity from nuclear (TWh)', axis=1)
pd.DataFrame(data)

,Entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,Afghanistan,2000,1.61,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,...,302.59,1.64,760.0,NaN,NaN,NaN,60.0,652230.0,33.939110,67.709953
1,Afghanistan,2001,4.07,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,...,236.89,1.74,730.0,NaN,NaN,NaN,60.0,652230.0,33.939110,67.709953
2,Afghanistan,2002,9.41,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,...,210.86,1.40,1030.0,NaN,NaN,179.43,60.0,652230.0,33.939110,67.709953
3,Afghanistan,2003,14.74,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,...,229.97,1.40,1220.0,NaN,8.83,190.68,60.0,652230.0,33.939110,67.709953
4,Afghanistan,2004,20.06,10.9,7.75,NaN,44.24,0.33,0.0,0.56,...,204.23,1.20,1030.0,NaN,1.41,211.38,60.0,652230.0,33.939110,67.709953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,Zimbabwe,2016,42.56,29.8,62.88,30000.0,81.90,3.50,0.0,3.32,...,3227.68,10.00,11020.0,NaN,0.76,1464.59,38.0,390757.0,-19.015438,29.154857
3645,Zimbabwe,2017,44.18,29.8,62.33,5570000.0,82.46,3.05,0.0,4.30,...,3068.01,9.51,10340.0,NaN,4.71,1235.19,38.0,390757.0,-19.015438,29.154857
3646,Zimbabwe,2018,45.57,29.9,82.53,10000.0,80.23,3.73,0.0,5.46,...,3441.99,9.83,12380.0,NaN,4.82,1254.64,38.0,390757.0,-19.015438,29.154857
3647,Zimbabwe,2019,46.78,30.1,81.40,250000.0,81.50,3.66,0.0,4.58,...,3003.66,10.47,11760.0,NaN,-6.14,1316.74,38.0,390757.0,-19.015438,29.154857


Can MCAR Methods be used?

In [6]:
data_no_countries = pd.read_table('./co2_emissions/emission_data_modified.csv', sep=',')
mt = MCARTest(method="little")
if mt.little_mcar_test(data_no_countries) > 0.05:
    print('Is MCAR')
else:
    print('Not MCAR')

Not MCAR


Data is not MCAR so KNN can't really be used, but it can be a solution to try with the dataset anyways

In [7]:
knn_imputer = KNNImputer(n_neighbors=1)
imputed_data = knn_imputer.fit_transform(data_no_countries)
pd.DataFrame(imputed_data, columns=columns[2:])

,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),Low-carbon electricity (% electricity),Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
0,1.61,6.2,9.22,20000.0,44.99,0.16,0.0,0.31,65.96,302.59,1.64,760.0,14.79,1.41,211.38,60.0,652230.0,33.94,67.71
1,4.07,7.2,8.86,130000.0,45.60,0.09,0.0,0.50,84.75,236.89,1.74,730.0,14.79,1.41,211.38,60.0,652230.0,33.94,67.71
2,9.41,8.2,8.47,3950000.0,37.83,0.13,0.0,0.56,81.16,210.86,1.40,1030.0,4.90,1.41,179.43,60.0,652230.0,33.94,67.71
3,14.74,9.5,8.09,25970000.0,36.66,0.31,0.0,0.63,67.02,229.97,1.40,1220.0,4.90,8.83,190.68,60.0,652230.0,33.94,67.71
4,20.06,10.9,7.75,3950000.0,44.24,0.33,0.0,0.56,62.92,204.23,1.20,1030.0,4.90,1.41,211.38,60.0,652230.0,33.94,67.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,42.56,29.8,62.88,30000.0,81.90,3.50,0.0,3.32,48.68,3227.68,10.00,11020.0,1.65,0.76,1464.59,38.0,390757.0,-19.02,29.15
3645,44.18,29.8,62.33,5570000.0,82.46,3.05,0.0,4.30,58.50,3068.01,9.51,10340.0,1.65,4.71,1235.19,38.0,390757.0,-19.02,29.15
3646,45.57,29.9,82.53,10000.0,80.23,3.73,0.0,5.46,59.41,3441.99,9.83,12380.0,1.65,4.82,1254.64,38.0,390757.0,-19.02,29.15
3647,46.78,30.1,81.40,250000.0,81.50,3.66,0.0,4.58,55.58,3003.66,10.47,11760.0,1.65,-6.14,1316.74,38.0,390757.0,-19.02,29.15


Strategy #1: Take the average by country of the other values for the NaNs, if the country is entirely NaN, take the global average

In [8]:
grouped = data.groupby('Entity')
print(grouped[columns[2:-2]])
# grouped_mean = grouped[columns[2:-2]].dropna(how='any').transform('mean')
# print(grouped_mean)
# data[columns[2:-2]] = data[columns[2:-2]].fillna(grouped_mean, inplace=True)
# pd.DataFrame(data)

Strategy #2: Take the average by year for NaNs

In [9]:
# data.replace(0, np.nan)
average_per_column = data.groupby('Year').mean(numeric_only=True)
pd.DataFrame(average_per_column)

,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%),Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),Low-carbon electricity (% electricity),Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),Value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Density\n(P/Km2),Land Area(Km2),Latitude,Longitude
Year,,,,,,,,,,,,,,,,,,,
2000,73.121059,58.063110,88.568231,2.572255e+07,34.489070,49.466453,13.430838,14.882326,36.630936,24297.358382,6.281462,125001.536500,10.577639,4.300719,7364.885385,225.191860,637743.680233,18.07687,14.744577
2001,73.646471,58.628659,83.596977,2.371608e+07,34.090819,50.793333,13.893554,14.545146,35.961647,24489.267733,6.186725,127215.009313,10.351389,3.365260,7244.108662,225.191860,637743.680233,18.07687,14.744577
2002,73.992573,59.192683,84.579457,2.476600e+07,33.999532,53.094386,13.946084,15.022807,36.324706,24897.487384,6.071988,128975.908500,10.204861,3.414129,7590.195696,225.191860,637743.680233,18.07687,14.744577
2003,74.564620,59.779878,84.987132,5.896115e+07,33.633450,56.026725,13.701566,15.180234,35.800588,25340.019709,6.073977,135309.799250,10.083611,4.039167,8826.220886,225.191860,637743.680233,18.07687,14.744577
2004,75.147895,60.332317,85.498837,2.559237e+07,33.458187,58.433392,14.346928,16.343977,35.960294,26004.756047,5.967018,141869.146625,10.391667,5.911538,10108.426709,225.191860,637743.680233,18.07687,14.744577
2005,75.560702,60.902744,86.348372,2.931891e+07,33.381053,61.181988,14.287651,17.125848,35.878176,26438.515000,5.792398,147313.503938,10.445833,5.158790,11035.733418,225.191860,637743.680233,18.07687,14.744577
2006,76.492632,61.474085,86.496279,2.984672e+07,33.078129,63.806257,14.432048,17.942632,35.281118,26400.794593,5.617602,151966.282312,10.567639,6.126561,12521.874969,225.191860,637743.680233,18.07687,14.744577
2007,76.913506,62.052108,99.734341,5.970303e+07,32.491329,67.671445,13.939226,18.377399,34.962558,26427.884598,5.439133,156387.900432,10.461111,5.792500,14112.831553,223.448276,630937.925287,18.36751,14.807169
2008,77.440862,62.603313,100.574806,3.237941e+07,32.580289,67.636936,13.802917,19.949595,35.571744,26541.192184,5.330751,157413.630988,10.841389,4.000813,15480.174161,223.448276,630937.925287,18.36751,14.807169
